# lab 12-4 stack RNN 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to"
           "collect wood and don't assign them tasks and work, but rather"
           "teach them to long for the endless immensity of the sea.")


In [3]:
char_set = list(set(sentence))
char_dic = {c:i for i,c in enumerate(char_set)}

In [4]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_lenth = 10
learning_rate = 0.1

In [5]:
dataX = []
dataY = []
for i in range(0,len(sentence)-sequence_lenth):
    x_str = sentence[i:i+sequence_lenth]
    y_str = sentence[i+1:i+sequence_lenth+1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [6]:
batch_size = len(dataX)

In [9]:
x_one_hot = tf.one_hot(dataX,num_classes)
y_one_hot = tf.one_hot(dataY,num_classes)
print(x_one_hot.shape,'\n',y_one_hot.shape)

(168, 10, 25) 
 (168, 10, 25)


In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=num_classes, input_shape = (sequence_lenth, x_one_hot.shape[2]),
                              return_sequences = True))
model.add(tf.keras.layers.LSTM(units = num_classes, return_sequences=True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation = 'softmax')))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer = tf.keras.optimizers.Adam(learning_rate),
              metrics = ['accuracy'])
model.fit(x_one_hot,y_one_hot,epochs = 50)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
lstm_6 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 25)            650       
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
6/6 [==============================] - 0s 2ms/step - loss: 3.0838 - accuracy: 0.1305
Epoch 2/50
6/6 [==============================] - 0s 2ms/step - loss: 2.8589 - accuracy: 0.1852
Epoch 3/50
6/6 [==============================] - 0s 2ms/step - loss: 2.7532 - accuracy: 0.1783
Epoch 4/50
6/6 [==============================] - 0s 2ms/step - loss: 2.4354 - accuracy: 0.2

In [15]:
results = model.predict(x_one_hot)
for j,result in enumerate(results):
    index = np.argmax(result,axis = 1)
    if j ==0:
        print(''.join([char_set[t] for t in index]),end = '')
    else:
        print(char_set[index[-1]],end = '')

t you want to butld a ship, don't drum up people together tocollect wood and don't dssign them tasks and work, but ratherteach them to long for the endless immensity of the sea.